# Medical Text

Medical Text Dataset [https://www.kaggle.com/datasets/chaitanyakck/medical-text/data]

In [1]:
import pandas as pd
import nltk
from spacy import displacy
import spacy
from transformers import pipeline
import pickle
import json
import os

In [2]:
df = pd.read_csv('data/train.dat', sep="\t", header=None)

In [3]:
df.rename(columns={0:'condition', 1:'abstract'}, inplace=True)
df.head()

,condition,abstract
0,4,Catheterization laboratory events and hospital...
1,5,Renal abscess in children. Three cases of rena...
2,2,Hyperplastic polyps seen at sigmoidoscopy are ...
3,5,Subclavian artery to innominate vein fistula a...
4,4,Effect of local inhibition of gamma-aminobutyr...


## Lexical Analysis
Lexical analysis consists of the operations of:
- **Sentence Splitting**: The technique aims to identify the beginning and end of a textual fragment (sentence or clause) with informative content, even if simple. To achieve this, it uses orthographic features of words (e.g., uppercase initial letters) and delimiters (e.g., punctuation).
- **Tokenization**: The goal of tokenization is to pinpoint the starting and ending positions of each token, whether it’s a word, a number, or a combination of symbols. As with sentence splitting, the process relies on orthographic features (e.g., initial capital letters) and delimiters (e.g., punctuation).
- **Lemmatization**: Post-tokenization techniques address the morphological analysis of word-tokens. Lemmatization identifies the base form (lemma) of inflected words, preserving their meaning and grammatical category. For example, the token _liked_ maps to the lemma *like*. This process minimizes lexical variation by consolidating different forms of the same word into a unified representation.
- **Stemming**: Like lemmatization, processes inflected forms but reduces them to their root, which may not correspond to a dictionary word. Unlike lemmatization, it focuses on inflections that create new words and may change the grammatical class, such as *probable* (adjective) stemming to *probably* (adverb).
- **POS Tagging**: Part-of-speech (POS) tagging assigns a grammatical category to each token, such as noun, verb, or adjective

It focuses on the main components of a text (words), and aims to recognize them in relation to the context in which they are used, such as sentences or clauses.

#### Funzioni utili
From now on there will be repetitive steps in order to check files existence prior to create each one: the following functions aims to simplify the process.

In [4]:
def check_existence(obj):
    """
    checks if we already stored the file
    Arg:
    obj = str name of the object
    """
    path = 'objects/'+obj+'.pkl'
    if os.path.exists(path):
        with open(path, 'rb') as file:
            file = pickle.load(file)
            return file
    else:
        return False

In [5]:
def save_step(name, obj):
    """
    Saves object to pickle and json files
    Args:
    name = 'obj_name'
    obj = object to save
    """

    with open('objects/'+name+'.pkl', 'wb') as file:
        pickle.dump(obj, file)

    with open('objects/'+name+'.json', 'w') as file:
        json.dump(obj, file)

In [6]:
lexical_df = df.copy() # a df to compute lexical analysis on

### Sentence Splitting, Word Tokenization, Lemmatization, Stemming and POS Tagging


In [7]:
# Load the SpaCy model
nlp = spacy.load("en_core_web_sm")

C:\Users\rubin\anaconda3\Lib\site-packages\spacy\util.py:910: UserWarning: [W095] Model 'en_core_web_sm' (3.8.0) was trained with spaCy v3.8.0 and may not be 100% compatible with the current version (3.7.5). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [8]:
if check_existence('sentences'): # if the first obj exists for sure we created the others(?)
    sentences = check_existence('sentences')
    words = check_existence('words')
    lemmatization = check_existence('lemmatization')
    stemming = check_existence('stemming')
    pos_tag = check_existence('pos_tag')

else:
    sentences = []
    words = []
    lemmatization = []

    stemming = []
    porterStemmer = nltk.PorterStemmer()

    pos_tag = []

    for record in df["abstract"]:
        doc = nlp(record)
        split_record = []
        tokenized_record = []
        lemmatized_record = []
        stemmed_record = []
        pos_tag_record = []

        for sentence in doc.sents:
            split_record.append([sentence.text]) # Sentence Splitting
            tokenized_sent = []
            lemmatized_sent = []
            stemmed_sent = []
            pos_tag_sent = []

            for token in sentence:
                word = token.text # Tokenization
                tokenized_sent.append(word)

                lemmatized_sent.append(token.lemma_) # Lemmatization

                stemmed_sent.append(porterStemmer.stem(word)) # Stemming

                pos_tag_sent.append((word,token.tag_)) # POS Tagging

                # print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_, token.shape_, token.is_alpha, token.is_stop)
            tokenized_record.append(tokenized_sent)
            lemmatized_record.append(lemmatized_sent)
            stemmed_record.append(stemmed_sent)
            pos_tag_record.append(pos_tag_sent)

        sentences.append(split_record)
        words.append(tokenized_record)
        lemmatization.append(lemmatized_record)
        stemming.append(stemmed_record)
        pos_tag.append(pos_tag_record)

    # Save each step to a file
    save_step('sentences',sentences)
    save_step('words',words)
    save_step('lemmatization',lemmatization)
    save_step('stemming',stemming)
    save_step('pos_tag',pos_tag)

In [9]:
lexical_df["sentences"] = sentences
lexical_df['words'] = words
lexical_df['lemmatization'] = lemmatization
lexical_df['stemming'] = stemming
lexical_df['pos_tag'] = pos_tag

In [10]:
lexical_df.head()

,condition,abstract,sentences,words,lemmatization,stemming,pos_tag
0,4,Catheterization laboratory events and hospital...,[[Catheterization laboratory events and hospit...,"[[Catheterization, laboratory, events, and, ho...","[[catheterization, laboratory, event, and, hos...","[[catheter, laboratori, event, and, hospit, ou...","[[(Catheterization, NN), (laboratory, NN), (ev..."
1,5,Renal abscess in children. Three cases of rena...,"[[Renal abscess in children.], [Three cases of...","[[Renal, abscess, in, children, .], [Three, ca...","[[Renal, abscess, in, child, .], [three, case,...","[[renal, abscess, in, children, .], [three, ca...","[[(Renal, NNP), (abscess, NN), (in, IN), (chil..."
2,2,Hyperplastic polyps seen at sigmoidoscopy are ...,[[Hyperplastic polyps seen at sigmoidoscopy ar...,"[[Hyperplastic, polyps, seen, at, sigmoidoscop...","[[hyperplastic, polyp, see, at, sigmoidoscopy,...","[[hyperplast, polyp, seen, at, sigmoidoscopi, ...","[[(Hyperplastic, JJ), (polyps, NNS), (seen, VB..."
3,5,Subclavian artery to innominate vein fistula a...,[[Subclavian artery to innominate vein fistula...,"[[Subclavian, artery, to, innominate, vein, fi...","[[subclavian, artery, to, innominate, vein, fi...","[[subclavian, arteri, to, innomin, vein, fistu...","[[(Subclavian, JJ), (artery, NN), (to, TO), (i..."
4,4,Effect of local inhibition of gamma-aminobutyr...,[[Effect of local inhibition of gamma-aminobut...,"[[Effect, of, local, inhibition, of, gamma, -,...","[[effect, of, local, inhibition, of, gamma, -,...","[[effect, of, local, inhibit, of, gamma, -, am...","[[(Effect, NN), (of, IN), (local, JJ), (inhibi..."


In [11]:
sentences[0][0]

['Catheterization laboratory events and hospital outcome with direct angioplasty for acute myocardial infarction To assess the safety of direct infarct angioplasty without antecedent thrombolytic therapy, catheterization laboratory and hospital events were assessed in consecutively treated patients with infarctions involving the left anterior descending (n = 100 patients), right (n = 100), and circumflex (n = 50) coronary arteries.']

### Stop-words Removal
Stop-words are common words that do not carry specific meaning, such as articles, prepositions, and conjunctions.
It is usually performed after lexical analysis to avoid inaccuracies in subsequent syntactic or semantic analyses.

In [12]:
# nltk.download('stopwords')

In [13]:
if check_existence('stopwords_removal'):
    stopwords_removal = check_existence('stopwords_removal')

else:
    stopwords = nltk.corpus.stopwords.words('english')

    stopwords_removal = []

    for record in lexical_df['sentences']:
        filtered_record = []
        for sentence in record:
            filtered_sentence = [word for word in sentence if word.lower() not in stopwords]
            filtered_record.append(filtered_sentence)

        stopwords_removal.append(filtered_record)

    save_step('stopwords_removal',stopwords_removal)

In [14]:
lexical_df["stopwords_removal"] = stopwords_removal

In [15]:
lexical_df.head()

,condition,abstract,sentences,words,lemmatization,stemming,pos_tag,stopwords_removal
0,4,Catheterization laboratory events and hospital...,[[Catheterization laboratory events and hospit...,"[[Catheterization, laboratory, events, and, ho...","[[catheterization, laboratory, event, and, hos...","[[catheter, laboratori, event, and, hospit, ou...","[[(Catheterization, NN), (laboratory, NN), (ev...",[[Catheterization laboratory events and hospit...
1,5,Renal abscess in children. Three cases of rena...,"[[Renal abscess in children.], [Three cases of...","[[Renal, abscess, in, children, .], [Three, ca...","[[Renal, abscess, in, child, .], [three, case,...","[[renal, abscess, in, children, .], [three, ca...","[[(Renal, NNP), (abscess, NN), (in, IN), (chil...","[[Renal abscess in children.], [Three cases of..."
2,2,Hyperplastic polyps seen at sigmoidoscopy are ...,[[Hyperplastic polyps seen at sigmoidoscopy ar...,"[[Hyperplastic, polyps, seen, at, sigmoidoscop...","[[hyperplastic, polyp, see, at, sigmoidoscopy,...","[[hyperplast, polyp, seen, at, sigmoidoscopi, ...","[[(Hyperplastic, JJ), (polyps, NNS), (seen, VB...",[[Hyperplastic polyps seen at sigmoidoscopy ar...
3,5,Subclavian artery to innominate vein fistula a...,[[Subclavian artery to innominate vein fistula...,"[[Subclavian, artery, to, innominate, vein, fi...","[[subclavian, artery, to, innominate, vein, fi...","[[subclavian, arteri, to, innomin, vein, fistu...","[[(Subclavian, JJ), (artery, NN), (to, TO), (i...",[[Subclavian artery to innominate vein fistula...
4,4,Effect of local inhibition of gamma-aminobutyr...,[[Effect of local inhibition of gamma-aminobut...,"[[Effect, of, local, inhibition, of, gamma, -,...","[[effect, of, local, inhibition, of, gamma, -,...","[[effect, of, local, inhibit, of, gamma, -, am...","[[(Effect, NN), (of, IN), (local, JJ), (inhibi...",[[Effect of local inhibition of gamma-aminobut...


# Syntax Analysis
Syntax analysis consists of:
- Shallow Parsing
- Deep Parsing

### Shallow Parsing
Syntactic parsing extends chunking by generating a parse tree. This tree organizes POS-tagging results as leaf nodes and syntactic structures (often chunks) as intermediate nodes, connected hierarchically without representing specific relationships.


In [16]:
syntax_df = df.copy() # a df to compute syntax analysis on

In [17]:
if check_existence('chunking'):
    chunking = check_existence('chunking')

else:
    # Define the grammar and the chunk parser
    grammar = "NP: {<NNP><NNP>}"
    cp = nltk.RegexpParser(grammar) # chunk parser

    # Apply chunking to each record
    chunking = []
    for record in lexical_df['pos_tag']:
        chunked_record = [cp.parse(sentence) for sentence in record]

        chunking.append(chunked_record)

    save_step('chunking',chunking)

In [18]:
# Save the chunking results into the dataframe
syntax_df["shallow_parsing"] = chunking

In [19]:
# Display the dataframe
syntax_df.head()

,condition,abstract,shallow_parsing
0,4,Catheterization laboratory events and hospital...,"[[(Catheterization, NN), (laboratory, NN), (ev..."
1,5,Renal abscess in children. Three cases of rena...,"[[(Renal, NNP), (abscess, NN), (in, IN), (chil..."
2,2,Hyperplastic polyps seen at sigmoidoscopy are ...,"[[(Hyperplastic, JJ), (polyps, NNS), (seen, VB..."
3,5,Subclavian artery to innominate vein fistula a...,"[[(Subclavian, JJ), (artery, NN), (to, TO), (i..."
4,4,Effect of local inhibition of gamma-aminobutyr...,"[[(Effect, NN), (of, IN), (local, JJ), (inhibi..."


In [20]:
# Display the chunking result for the first record
print(syntax_df['shallow_parsing'][0][4])
#syntax_df['chunking'][0][4]

(S
  There/EX
  was/VBD
  one/CD
  in/IN
  -/HYPH
  laboratory/NN
  death/NN
  (/-LRB-
  shock/NN
  patient/NN
  with/IN
  infarction/NN
  of/IN
  the/DT
  left/JJ
  anterior/JJ
  descending/VBG
  coronary/JJ
  artery/NN
  )/-RRB-
  ./.)


In [21]:
# To draw the parse tree
#syntax_df['chunking'][1][1].draw()

### Deep Parsing
Differently from _Shallow parsing_, _Deep parsing_ aims to infer dependency relationships between nodes.
The result is a dependency graph which relates words that are syntactically linked.

In [22]:
if check_existence('deep_parsing'):
    deep_parsing = check_existence('deep_parsing')

else:
    deep_parsing = []
    for sentences in syntax_df["abstract"]:
        sentence_dep = []
        doc = nlp(sentences)
        for token in doc:
            sentence_dep.append((str(token.text), str(token.dep_), str(token.head.text), str([child for child in token.children])))
            # creates a tuple containing the token, dependency nature, head and all dependents of the token
        deep_parsing.append(sentence_dep)

    save_step('deep_parsing',deep_parsing)

In [23]:
syntax_df["deep_parsing"] = deep_parsing

In [24]:
syntax_df.head()

,condition,abstract,shallow_parsing,deep_parsing
0,4,Catheterization laboratory events and hospital...,"[[(Catheterization, NN), (laboratory, NN), (ev...","[(Catheterization, compound, events, []), (lab..."
1,5,Renal abscess in children. Three cases of rena...,"[[(Renal, NNP), (abscess, NN), (in, IN), (chil...","[(Renal, nsubj, abscess, []), (abscess, ROOT, ..."
2,2,Hyperplastic polyps seen at sigmoidoscopy are ...,"[[(Hyperplastic, JJ), (polyps, NNS), (seen, VB...","[(Hyperplastic, amod, polyps, []), (polyps, ns..."
3,5,Subclavian artery to innominate vein fistula a...,"[[(Subclavian, JJ), (artery, NN), (to, TO), (i...","[(Subclavian, amod, artery, []), (artery, nsub..."
4,4,Effect of local inhibition of gamma-aminobutyr...,"[[(Effect, NN), (of, IN), (local, JJ), (inhibi...","[(Effect, ROOT, Effect, [of, :, study, .]), (o..."


In [25]:
# Seleziona la prima riga del dataframe
sentence = syntax_df["abstract"].iloc[0]

# Analizza la frase
doc = nlp(sentence)

# Visualizza il grafico di dipendenze
displacy.render(doc, style="dep", jupyter=True)

## Semantic Analysis

Semantic analysis aims to extract the meaning of a text, focusing on the relationships between entities and the context in which they appear.

### Entity Extraction

Entity extraction identifies named entities in a text, such as people, organizations, or locations.

In [26]:
semantic_df = df.copy() # a df to compute semantic analysis on

This code extracts named entities from the "abstract" column of a DataFrame called syntax_df using a NLP model.
For each record, it collects the entities and their labels, storing them in a list of tuples.

In [27]:
pipe = pipeline("token-classification", model="Clinical-AI-Apollo/Medical-NER", aggregation_strategy='simple')

In [28]:
# if the entities are already extracted, load them from the pickle file
if check_existence('entities'):
    entities = check_existence('entities')

else:
    entities = []
    for record in semantic_df["abstract"]:
        result = pipe(record)

        record_entities = [(entity['word'], entity['entity_group']) for entity in result]

        entities.append(record_entities)

    save_step('entities',entities)

In [29]:
semantic_df["entities"] = entities

To get a sense of the entities extracted from the text, we can display the unique entities found in the dataset.

In [30]:
# Flatten the list of entities and extract only the labels
all_labels = pd.Series([label for record in semantic_df["entities"] for _, label in record])

# Perform value counts on the labels
all_labels.value_counts()

DETAILED_DESCRIPTION      100224
DIAGNOSTIC_PROCEDURE       74383
SIGN_SYMPTOM               68825
LAB_VALUE                  68092
DISEASE_DISORDER           61689
BIOLOGICAL_STRUCTURE       55139
THERAPEUTIC_PROCEDURE      27167
MEDICATION                 19053
DATE                       15674
DURATION                    5689
DOSAGE                      5519
AGE                         3286
CLINICAL_EVENT              2792
DISTANCE                    2380
SEVERITY                    2013
NONBIOLOGICAL_LOCATION      1819
HISTORY                     1763
SEX                         1451
FAMILY_HISTORY               426
AREA                         139
ADMINISTRATION               129
COREFERENCE                  122
OTHER_ENTITY                  15
PERSONAL_BACKGROUND           13
VOLUME                         9
TIME                           4
Name: count, dtype: int64

In [31]:
semantic_df.head()

,condition,abstract,entities
0,4,Catheterization laboratory events and hospital...,"[(Catheterization laboratory events, DIAGNOSTI..."
1,5,Renal abscess in children. Three cases of rena...,"[(Renal, BIOLOGICAL_STRUCTURE), (abscess, DISE..."
2,2,Hyperplastic polyps seen at sigmoidoscopy are ...,"[(Hyperplastic polyps, SIGN_SYMPTOM), (sigmoid..."
3,5,Subclavian artery to innominate vein fistula a...,"[(Subclavian artery to innominate vein, BIOLOG..."
4,4,Effect of local inhibition of gamma-aminobutyr...,"[(dorsomedial hypothalamus, BIOLOGICAL_STRUCTU..."


In [32]:
def display_entities(pipe, document):
    """
    Mostra le entità riconosciute utilizzando displacy.render().

    Parametri:
        pipe: Pipeline Hugging Face per il riconoscimento delle entità.
        document (str): Testo da processare.

    Ritorna:
        - Visualizzazione delle entità tramite displacy.
        - Lista di entità e relative etichette.
    """
    # Esegui la pipeline sul documento
    result = pipe(document)

    # Crea un modello SpaCy vuoto per gestire il documento
    nlp = spacy.blank("en")
    doc = nlp.make_doc(document)

    # Aggiungi le entità manualmente
    ents = []
    for entity in result:
        start_char = entity['start']
        end_char = entity['end']
        label = entity['entity_group']
        # Crea manualmente uno span
        span = doc.char_span(start_char, end_char, label=label, alignment_mode="expand")
        if span is not None:
            ents.append(span)

    # Assegna le entità al documento
    doc.ents = ents

    # Visualizza con displacy
    displacy.render(doc, style='ent', jupyter=True)

In [33]:
display_entities(pipe, semantic_df["abstract"].iloc[1])

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [34]:
semantic_df.head()

,condition,abstract,entities
0,4,Catheterization laboratory events and hospital...,"[(Catheterization laboratory events, DIAGNOSTI..."
1,5,Renal abscess in children. Three cases of rena...,"[(Renal, BIOLOGICAL_STRUCTURE), (abscess, DISE..."
2,2,Hyperplastic polyps seen at sigmoidoscopy are ...,"[(Hyperplastic polyps, SIGN_SYMPTOM), (sigmoid..."
3,5,Subclavian artery to innominate vein fistula a...,"[(Subclavian artery to innominate vein, BIOLOG..."
4,4,Effect of local inhibition of gamma-aminobutyr...,"[(dorsomedial hypothalamus, BIOLOGICAL_STRUCTU..."


### Relation Extraction

Relation extraction identifies the connections between entities in a text, such as the subject, object, and verb of a sentence.

This process is more complex than entity extraction, as it requires understanding the syntactic structure of the text to infer relationships between entities.

1. **Iterate through sentences**:
   Process each sentence (`doc`) and its entities, storing entities in a set for faster lookup (`entities_set`).

2. **Extract relations from tokens**:
   Loop through tokens with dependencies like "ROOT" or "VERB" to identify:
   - **Subjects**: Found in children with dependencies like "nsubj" or "agent."
   - **Objects**: Found in children with dependencies like "dobj" or "pobj."

3. **Record relations**:
   - Direct relations: `(subject, verb, object).`
   - Prepositional relations: Handle `prep` and `pobj` to form `(subj, verb_prep, obj)` or similar.

4. **Handle conjunctions**:
   Add relations involving conjunctive tokens (`conj`).



In [35]:
if check_existence('relations'):
    all_relations = check_existence('relations')
else:
    # Relations Extraction
    all_relations = []

    for deep_parsing, sentence_entities in zip(syntax_df['deep_parsing'], semantic_df["entities"]):
        sentence_relations = []
        entities_set = {ent_text for ent_text, _ in sentence_entities}  # Use a set for faster lookup

        # Iterate through parsed tokens in the deep_parsing list
        for token_data in deep_parsing:
            token_text, dep, head_text, children = token_data

            # Identify verbs or root tokens
            if dep in ("ROOT", "VERB", "aux", "auxpass"):
                # Find the subject
                subj = None
                for child in children:
                    for ent_text in entities_set:
                        if ent_text in child or child in ent_text:
                            subj = ent_text
                            break

                # Find the object
                obj = None
                for child_data in deep_parsing:
                    child_text, child_dep, child_head_text, _ = child_data
                    if child_head_text == token_text and child_dep in ("dobj", "pobj", "iobj", "attr", "acomp"):
                        for ent_text in entities_set:
                            if ent_text in child_text or child_text in ent_text:
                                obj = ent_text
                                break

                # Record direct relations
                if subj and obj:
                    sentence_relations.append((subj, token_text, obj))

                # Handle prepositional relations
                for child_data in deep_parsing:
                    child_text, child_dep, child_head_text, grand_children = child_data
                    if child_head_text == token_text and child_dep == "prep":
                        for grandchild_data in deep_parsing:
                            grandchild_text, grandchild_dep, grandchild_head_text, _ = grandchild_data
                            if grandchild_head_text == child_text and grandchild_dep == "pobj":
                                for ent_text in entities_set:
                                    if ent_text in grandchild_text or grandchild_text in ent_text:
                                        if subj:
                                            sentence_relations.append((subj, f"{token_text}_{child_text}", ent_text))
                                        elif obj:  # Use the object if no subject
                                            sentence_relations.append((obj, f"{token_text}_{child_text}", ent_text))

                # Handle conjunctive subjects and objects
                for child_data in deep_parsing:
                    child_text, child_dep, child_head_text, _ = child_data
                    if child_head_text == token_text and child_dep == "conj" and subj:
                        sentence_relations.append((subj, token_text, child_text))

        # Post-process to remove duplicates
        sentence_relations = list(set(sentence_relations))

        # Add relations to the main list
        all_relations.append(sentence_relations)

    # Save the relations to a file
    save_step('relations', all_relations)

In [36]:
# Add the relations list as a column to semantic_df
semantic_df['relations'] = [relations if relations else pd.NA for relations in all_relations]

In [37]:
# Display the updated dataframe
semantic_df.head()

,condition,abstract,entities,relations
0,4,Catheterization laboratory events and hospital...,"[(Catheterization laboratory events, DIAGNOSTI...","[(hospital outcome, assessed_in, s), (hospital..."
1,5,Renal abscess in children. Three cases of rena...,"[(Renal, BIOLOGICAL_STRUCTURE), (abscess, DISE...","[(surgical procedure, were, associated), (S. a..."
2,2,Hyperplastic polyps seen at sigmoidoscopy are ...,"[(Hyperplastic polyps, SIGN_SYMPTOM), (sigmoid...","[(other findings, had_at, oid), (other finding..."
3,5,Subclavian artery to innominate vein fistula a...,"[(Subclavian artery to innominate vein, BIOLOG...","[(innominate vein, report, suggest), (innomina..."
4,4,Effect of local inhibition of gamma-aminobutyr...,"[(dorsomedial hypothalamus, BIOLOGICAL_STRUCTU...","[(2 to 2.5 hr, Effect_of, dorsomedial hypothal..."


In [38]:
# Filter and display only the tuples where the label is BIOLOGICAL_STRUCTURE
semantic_df['entities'].apply(lambda x: [(text, label) for text, label in x if label == 'MEDICATION'])

0                                                       []
1                              [(Antibiotics, MEDICATION)]
2                                                       []
3                                                       []
4        [(nipecotic acid, MEDICATION), (KCl, MEDICATIO...
                               ...                        
14433                                                   []
14434                                                   []
14435                [(platelet concentrates, MEDICATION)]
14436                                [(renal, MEDICATION)]
14437       [(heparin, MEDICATION), (heparin, MEDICATION)]
Name: entities, Length: 14438, dtype: object

### TF_IDF

In [39]:
from sklearn.feature_extraction.text import TfidfVectorizer

The following vectorizer takes a sequence of byte strings, converts it into lowercase, extracts unigrams, and calculates a TF-IDF score. It contains stop words from english vocabulary, the n-grams that occur in more than 60% of documents or in less than 10% of documents will be ignored.

In [40]:
vectorizer = TfidfVectorizer(input='content', use_idf=True, lowercase=True,
analyzer='word', ngram_range=(1, 1), stop_words='english', vocabulary=None, min_df=0.10, max_df=0.60)

In [41]:
tfidf_matrix = vectorizer.fit_transform(df['abstract'].values)
print(f"Matrix dimension: {tfidf_matrix.shape}")

Matrix dimension: (14438, 62)


In [42]:
feature_names = vectorizer.get_feature_names_out()
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), index=df.index, columns=feature_names)

In [43]:
tfidf_df.head()

,10,11,12,15,20,acute,age,analysis,associated,blood,...,therapy,time,treated,treatment,tumor,use,used,using,year,years
0,0.151755,0.0,0.0,0.000000,0.0,0.176468,0.164997,0.0,0.000000,0.000000,...,0.166472,0.0,0.170159,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.462777
1,0.204264,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.199219,0.000000,...,0.224074,0.0,0.000000,0.000000,0.248062,0.000000,0.000000,0.000000,0.0,0.207635
2,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
3,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.278452,0.000000,0.000000,0.0,0.000000
4,0.136076,0.0,0.0,0.159619,0.0,0.000000,0.000000,0.0,0.132715,0.151375,...,0.000000,0.0,0.000000,0.124461,0.000000,0.000000,0.150682,0.152792,0.0,0.000000


In [44]:
tfidf_df.loc['00_Document Frequency'] = (tfidf_df > 0).sum()

In [45]:
tfidf_df.tail()

,10,11,12,15,20,acute,age,analysis,associated,blood,...,therapy,time,treated,treatment,tumor,use,used,using,year,years
14434,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.00000,...,0.000000,0.000000,0.0,0.0,0.0,0.00000,0.0,0.665659,0.0,0.0
14435,0.252997,0.450696,0.139863,0.0,0.0,0.147099,0.0,0.0,0.0,0.14072,...,0.138766,0.000000,0.0,0.0,0.0,0.14881,0.0,0.000000,0.0,0.0
14436,0.182328,0.000000,0.604771,0.0,0.0,0.000000,0.0,0.0,0.0,0.00000,...,0.200009,0.410164,0.0,0.0,0.0,0.00000,0.0,0.000000,0.0,0.0
14437,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.00000,...,0.000000,0.000000,0.0,0.0,0.0,0.00000,0.0,0.000000,0.0,0.0
00_Document Frequency,2639.000000,1590.000000,1984.000000,1654.0,1709.0,1700.000000,2085.0,1601.0,2821.0,1948.00000,...,2031.000000,1884.000000,1902.0,3323.0,1479.0,1639.00000,1975.0,1894.000000,1915.0,2524.0


In [68]:
mapping = {
    'condition': {1: "neoplasm",
                  2: "digestive system disease",
                  3: "nervous system disease",
                  4: "cardiovascular disease",
                  5: "general pathological conditions"}
}

semantic_df.replace(mapping, inplace = True)

In [69]:
semantic_df.head()

,condition,abstract,entities,relations
0,cardiovascular disease,Catheterization laboratory events and hospital...,"[(Catheterization laboratory events, DIAGNOSTI...","[(hospital outcome, assessed_in, s), (hospital..."
1,general pathological conditions,Renal abscess in children. Three cases of rena...,"[(Renal, BIOLOGICAL_STRUCTURE), (abscess, DISE...","[(surgical procedure, were, associated), (S. a..."
2,digestive system disease,Hyperplastic polyps seen at sigmoidoscopy are ...,"[(Hyperplastic polyps, SIGN_SYMPTOM), (sigmoid...","[(other findings, had_at, oid), (other finding..."
3,general pathological conditions,Subclavian artery to innominate vein fistula a...,"[(Subclavian artery to innominate vein, BIOLOG...","[(innominate vein, report, suggest), (innomina..."
4,cardiovascular disease,Effect of local inhibition of gamma-aminobutyr...,"[(dorsomedial hypothalamus, BIOLOGICAL_STRUCTU...","[(2 to 2.5 hr, Effect_of, dorsomedial hypothal..."


## DATABASE

This is an image of the database schema we are going to create.

<div style="text-align: center;">
    <img src="images/er_schema.png" alt="Database Schema" width="700"/>
</div>

### Data Definition Language

In [46]:
CREATE_MEDICATIONS = """
DROP TABLE IF EXISTS MEDICATIONS;

CREATE TABLE MEDICATIONS(
id SERIAL PRIMARY KEY,
name VARCHAR NOT NULL,
dosage VARCHAR NOT NULL
);
"""

# Save to file
with open('database/ddl/create_medications.sql', 'w') as file:
    file.write(CREATE_MEDICATIONS)

In [47]:
CREATE_THERAPIES = """
DROP TABLE IF EXISTS THERAPIES;

CREATE TABLE THERAPIES(
id SERIAL PRIMARY KEY,
name VARCHAR NOT NULL
"""

# Save to file
with open('database/ddl/create_therapies.sql', 'w') as file:
    file.write(CREATE_THERAPIES)

In [48]:
CREATE_DISEASES = """
DROP TABLE IF EXISTS DISEASES;

CREATE TABLE DISEASES(
id SERIAL PRIMARY KEY,
name VARCHAR NOT NULL,
condition VARCHAR NOT NULL
);
"""

# Save to file
with open('database/ddl/create_diseases.sql', 'w') as file:
    file.write(CREATE_DISEASES)

In [49]:
CREATE_SYMPTOMS = """
DROP TABLE IF EXISTS SYMPTOMS;

CREATE TABLE SYMPTOMS(
id SERIAL PRIMARY KEY,
name VARCHAR NOT NULL
);
"""

# Save to file
with open('database/ddl/create_symptoms.sql', 'w') as file:
    file.write(CREATE_SYMPTOMS)

In [50]:
CREATE_DIAGNOSTIC_TESTS = """
DROP TABLE IF EXISTS DIAGNOSTIC_TESTS;

CREATE TABLE DIAGNOSTIC_TESTS(
id SERIAL PRIMARY KEY,
name VARCHAR NOT NULL,
labValue NUMERIC NOT NULL
);
"""

# Save to file
with open('database/ddl/create_diagnostic_tests.sql', 'w') as file:
    file.write(CREATE_DIAGNOSTIC_TESTS)

In [51]:
CREATE_BIOLOGICAL_STRUCTURES = """
DROP TABLE IF EXISTS BIOLOGICAL_STRUCTURES;

CREATE TABLE BIOLOGICAL_STRUCTURES(
id SERIAL PRIMARY KEY,
name VARCHAR NOT NULL,
);
"""

# Save to file
with open('database/ddl/create_biological_structures.sql', 'w') as file:
    file.write(CREATE_BIOLOGICAL_STRUCTURES)

In [52]:
# entità associative
CREATE_PRESCRIPTIONS = """
DROP TABLE IF EXISTS PRESCRIPTIONS;

CREATE TABLE PRESCRIPTIONS(
id SERIAL PRIMARY KEY,
medicationID INT NOT NULL,
therapyID INT NOT NULL,
FOREIGN KEY (medicationID) REFERENCES MEDICATIONS(id),
FOREIGN KEY (therapyID) REFERENCES THERAPIES(id)
"""

# Save to file
with open('database/ddl/create_prescriptions.sql', 'w') as file:
    file.write(CREATE_PRESCRIPTIONS)

In [53]:
CREATE_TREATMENTS = """
DROP TABLE IF EXISTS TREATMENTS;

CREATE TABLE TREATMENTS(
id SERIAL PRIMARY KEY,
diseaseID INT NOT NULL,
therapyID INT NOT NULL,
duration varchar,
FOREIGN KEY (diseaseID) REFERENCES DISEASES(id),
FOREIGN KEY (therapyID) REFERENCES THERAPIES(id)
);
"""

# Save to file
with open('database/ddl/create_treatments.sql', 'w') as file:
    file.write(CREATE_TREATMENTS)

In [54]:
CREATE_MANIFESTATIONS = """
DROP TABLE IF EXISTS MANIFESTATIONS;

CREATE TABLE MANIFESTATIONS(
id SERIAL PRIMARY KEY,
diseaseID INT NOT NULL,
symptomID INT NOT NULL,
severity VARCHAR,
FOREIGN KEY (diseaseID) REFERENCES DISEASES(id),
FOREIGN KEY (symptomID) REFERENCES SYMPTOMS(id)
);
"""

# Save to file
with open('database/ddl/create_manifestations.sql', 'w') as file:
    file.write(CREATE_MANIFESTATIONS)

In [55]:
CREATE_DIAGNOSIS = """
DROP TABLE IF EXISTS DIAGNOSIS;

CREATE TABLE DIAGNOSIS(
id SERIAL PRIMARY KEY,
diseaseID INT NOT NULL,
diagnosticTestID INT NOT NULL,
FOREIGN KEY (diseaseID) REFERENCES DISEASES(id),
FOREIGN KEY (diagnosticTestID) REFERENCES DIAGNOSTIC_TESTS(id)
)"""

# Save to file
with open('database/ddl/create_diagnosis.sql', 'w') as file:
    file.write(CREATE_DIAGNOSIS)

In [56]:
CREATE_INVOLVEMENT = """
DROP TABLE IF EXISTS INVOLVEMENT;

CREATE TABLE INVOLVEMENTS(
id SERIAL PRIMARY KEY,
diseaseID INT NOT NULL,
biologicalStrID INT NOT NULL,
FOREIGN KEY (diseaseID) REFERENCES DISEASES(id),
FOREIGN KEY (biologicalStrID) REFERENCES BIOLOGICAL_STRUCTURES(id)
"""

# Save to file
with open('database/ddl/create_involvement.sql', 'w') as file:
    file.write(CREATE_INVOLVEMENT)

In [57]:
# List of DDL files to concatenate
ddl_files = [
    'database/ddl/create_medications.sql',
    'database/ddl/create_therapies.sql',
    'database/ddl/create_diseases.sql',
    'database/ddl/create_symptoms.sql',
    'database/ddl/create_diagnostic_tests.sql',
    'database/ddl/create_biological_structures.sql',
    'database/ddl/create_prescriptions.sql',
    'database/ddl/create_treatments.sql',
    'database/ddl/create_manifestations.sql',
    'database/ddl/create_diagnosis.sql',
    'database/ddl/create_involvement.sql'
]

# Output file
output_file = 'database/ddl/create_all_tables.sql'

# Concatenate the contents of all DDL files into the output file
with open(output_file, 'w') as outfile:
    for ddl_file in ddl_files:
        with open(ddl_file, 'r') as infile:
            outfile.write(infile.read())

### Data Manipulation Language

In [58]:
# Extract MEDICATION and DOSAGE entities
medications = []
dosages = []

for entities in semantic_df['entities']:
    medication = None
    dosage = None
    for text, label in entities:
        if label == 'MEDICATION':
            medication = text
        elif label == 'DOSAGE':
            dosage = text
    if medication:
        medications.append(medication)
        dosages.append(dosage)

# Prepare data for insertion into the MEDICATIONS table
medications_data = list(zip(medications, dosages))

# Display the data to be inserted
medications_data

[('Antibiotics', None),
 ('nipecotic acid', '88 pmol/250 nl)'),
 ('opioid analgesia', None),
 ('corticosteroids', None),
 ('digoxin', None),
 ('Cimetidine', 'every 8, 6, or 4 h'),
 ('acyclovir', '30 mg per kilogram per day'),
 ('dCF', None),
 ('isoflurane', None),
 ('chemotherapy', None),
 ('progesterone', None),
 ('anticonvulsant medication', None),
 ('vitamin B6', None),
 ('acetic acid', None),
 ('anti', None),
 ('chemotherapy', None),
 ('local anesthetic', '46'),
 ('nimodipine', None),
 ('Amiodarone', None),
 ('chemotherapy', None),
 ('diphenylcyclopropenone', None),
 ('Plaquenil', None),
 ('Magnesium sulfate', '20 mL/kg'),
 ('rIL-2', '25,000 U/kg IP every 8 hours'),
 ('immunosuppressive drugs', None),
 ('apneic', '100'),
 ('vaso', None),
 ('405', '10 mg/kg orally'),
 ('corticosteroids', None),
 ('morphine', '1-2-mg iv boluses'),
 ('chemotherapy', None),
 ('baclofen', 'micrograms'),
 ('cytostatic combinations', 'six'),
 ('mexiletine', None),
 ('corticosteroid', None),
 ('phenytoin',

In [59]:
# MEDICATIONS
insert_medications = []
for medication, dosage in medications_data:
    if dosage is None:
        query = f"INSERT INTO MEDICATIONS (name, dosage) VALUES ('{medication}', NULL);"
    else:
        query = f"INSERT INTO MEDICATIONS (name, dosage) VALUES ('{medication}', '{dosage}');"
    insert_medications.append(query)

# Save the queries to a file
with open('database/dml/insert_medications.sql', 'w') as file:
    for query in insert_medications:
        file.write(query + '\n')

In [60]:
# THERAPIES
therapies = []

for entities in semantic_df['entities']:
    for text, label in entities:
        if label == 'THERAPEUTIC_PROCEDURE':
            therapies.append(text)

# Prepare data for insertion into the THERAPIES table
therapies

['angioplasty',
 'infarct',
 'angioplasty',
 'thrombolytic therapy',
 'cardioversion',
 'cardiopulmonary resuscitation',
 'dopamine',
 '-aortic balloon pump support',
 'surgery',
 'surgical procedure',
 'polypectomy',
 'hemodialysis catheter',
 'hemodialysis catheters',
 'catheterization',
 'fluoroscopy',
 'microdialysis',
 'microdialysis',
 'microdialysis',
 'perfusion',
 'perfusion',
 'microdialysis',
 'perfusion',
 'epidural catheterization',
 'epidural catheterization',
 'epidural catheters',
 'catheter removal',
 'surgery',
 'decompression',
 'Catheters',
 'replaced',
 'epidural catheterization',
 'treatment',
 'stinal',
 'tracheostomy',
 'o',
 'flap',
 'resection',
 'stinal',
 'tracheostomy',
 'stinal',
 'tracheostomy',
 'laryngoesophagectomy',
 'resection',
 'manubrectomy',
 'stoma',
 'tracheal stoma',
 'stoma',
 'stinal',
 'tracheostomy',
 'laryngoesophagectomy',
 'resection',
 'implantable cardioverter defibrillator',
 'cardioverter defibrillator',
 'D',
 'therapy',
 'endoscop

In [61]:
# Symptom entities
symptoms = []

for entities in semantic_df['entities']:
    for text, label in entities:
        if label == 'SIGN_SYMPTOM':
            symptoms.append(text)

symptoms

['infarction',
 'infarction',
 'infarction',
 'infarction',
 'hypotension',
 'infarction',
 'infarction',
 'infarction',
 'death',
 'infarction',
 'abscesses',
 'Fever',
 'pain',
 'leukocytosis',
 'abscesses',
 'Reflux',
 'Hyperplastic polyps',
 'adenomas',
 'Asym',
 'Polyps',
 'polyps',
 'rectosigm',
 'polyps',
 'rectosigmoid adenoma',
 'hyperplastic polyps',
 'other findings',
 'adenoma',
 'tosigm',
 'polyps',
 'adenomas',
 'Proximal',
 'adenomas',
 'adenomas',
 'fistula',
 's',
 'fistula',
 'tachycardia',
 'o-excitatory',
 'tachycardia',
 'tachycardia',
 'infection',
 'infection',
 'infection',
 'infection',
 'infection',
 'infection',
 'infections',
 'infections',
 'infection',
 'infection',
 'invasion',
 'necrosis',
 'rupture',
 'skin breakdown',
 'bleeding',
 'response',
 'secondary inflammatory component',
 'monoclonal',
 'aberrant phenotypes',
 'asthma',
 'asthma',
 'wheeze',
 'asthma',
 'wheeze',
 'wheeze',
 'asthma',
 'left ventricular function',
 'antibodies',
 'HBsAg',
 'HB

In [62]:
# Insert the symptoms into the SYMPTOMS table
insert_symptoms = []

for symptom in symptoms:
    query = f"INSERT INTO SYMPTOMS (name) VALUES ('{symptom}');"
    insert_symptoms.append(query)

# Save the queries to a file
with open('database/dml/insert_symptoms.sql', 'w') as file:
    for query in insert_symptoms:
        file.write(query + '\n')

In [63]:
# Diseases entities
diseases = []

for entities in semantic_df['entities']:
    for text, label in entities:
        if label == 'DISEASE_DISORDER':
            diseases.append(text)

diseases

['myocardial infarction',
 's',
 'multivessel disease',
 'Cardiogenic shock',
 'abscess',
 'appendicitis',
 'peritonitis',
 'Wilms tumor',
 'organism',
 'Escherichia coli',
 'Staphylococcus aureus',
 'E. coli',
 'S. aureus',
 'matic',
 'oid',
 'tous',
 'arteriovenous fistula',
 'Infection',
 'carcinoma',
 'carcinoma',
 'efactive fibroinflammatory lesion',
 'efactive fibroinflammatory lesion',
 'fibroinflammatory process',
 'fibroinflammatory lesion',
 'inflammatory lesion',
 'inflammatory',
 'autotopagnosia',
 'schema',
 'Alzheimer',
 's disease',
 'coronary artery disease',
 'coronary artery disease',
 'arrhythmias',
 'cardiac arrest',
 'ventricular tachycardia',
 'coronary',
 '-related mucosal damage',
 'stress-related mucosal damage',
 'SRMD',
 'SRMD',
 'mucosal damage',
 'hemorrhage',
 'hepatitis B virus infection',
 'hepatitis B',
 'chronic liver disease',
 'disease',
 'hepatitis B virus infection',
 'chronic liver disease',
 'B infection',
 'herpes simplex virus infection',
 'her

In [64]:
insert_diseases = []

for disease in diseases:
    query = f"INSERT INTO DISEASES (name) VALUES ('{disease}');"
    insert_diseases.append(query)

# Save the queries to a file
with open('database/dml/insert_diseases.sql', 'w') as file:
    for query in insert_diseases:
        file.write(query + '\n')

In [65]:
# Diagnostic Test entities
diagnostic_tests = []

for entities in semantic_df['entities']:
    for text, label in entities:
        if label == 'DIAGNOSTIC_PROCEDURE':
            diagnostic_tests.append(text)

diagnostic_tests

['Catheterization laboratory events',
 'hospital outcome',
 'catheterization laboratory',
 'hospital',
 'grade flow',
 'catheterization laboratory',
 'urinalysis',
 'urine culture',
 'sigmoidoscopy',
 'colonoscopy',
 'sigmoidoscopy',
 'endo',
 'sigmoidoscopy',
 'colonoscopy',
 'sigmoid',
 'gamma-aminobutyric acid',
 'GABA',
 'heart rate',
 'blood pressure',
 'GABA',
 'taurine',
 'heart rate',
 'arterial pressure',
 'GABA',
 'aspartate',
 'glutamate',
 'taurine',
 'glycine',
 'alanine',
 'heart rate',
 'arterial pressure',
 'GABA',
 'heart rate',
 'GABA',
 'histopathological',
 'Immunohistochemical studies',
 'Pointing',
 'pointed',
 'asthma symptoms',
 'spirometry',
 'Clinical outcome',
 'left ventricular (LV) ejection fractions',
 'LV ejection fraction',
 'survival',
 'acid',
 'serum',
 'Anti-pre-S antibodies',
 'enzyme immunoassays',
 'monoclonal antibodies',
 's',
 '-pre-S antibodies',
 'Anti-pre-S',
 'pre-S',
 'pre-S1',
 '-S2 proteins',
 'infectious',
 'developing',
 'developing',


In [66]:
insert_diagnostic_tests = []

for diagnostic_test in diagnostic_tests:
    query = f"INSERT INTO DIAGNOSTIC_TESTS (name) VALUES ('{diagnostic_test}');"
    insert_diagnostic_tests.append(query)

with open('database/dml/insert_diagnostic_tests.sql', 'w') as file:
    for query in insert_diagnostic_tests:
        file.write(query + '\n')

### Query Language

In [67]:
semantic_df[semantic_df['entities'].apply(
    lambda entities: any(label == 'MEDICATION' for _, label in entities) and any(label == 'DOSAGE' for _, label in entities)
)]

,condition,abstract,entities,relations
4,4,Effect of local inhibition of gamma-aminobutyr...,"[(dorsomedial hypothalamus, BIOLOGICAL_STRUCTU...","[(2 to 2.5 hr, Effect_of, dorsomedial hypothal..."
11,5,Stress-related mucosal damage: review of drug ...,"[(Stress, DETAILED_DESCRIPTION), (-related muc...","[(stress-related mucosal damage, been_Of, H2-r..."
13,5,A controlled trial comparing vidarabine with a...,"[(herpes simplex virus infection, DISEASE_DISO...","[(virologically confirmed, was_of, disease), (..."
46,5,The treatment of chondrodermatitis nodularis w...,"[(chondrodermatitis nodularis, DISEASE_DISORDE...","[(16 months, treatment_of, chondrodermatitis n..."
65,5,Does indomethacin alter the hemodynamic respon...,"[(hemodynamic, DIAGNOSTIC_PROCEDURE), (magnesi...","[(0.8 and 0.9 of timed gestation, was, 36 +/- ..."
...,...,...,...,...
14378,4,Avoiding interpretive pitfalls when assessing ...,"[(arrhythmia, SIGN_SYMPTOM), (myocardial infar...","[(greater than or equal to 5 runs of, performe..."
14396,5,Treatment of chronic tinea pedis (athlete's fo...,"[(chronic, DETAILED_DESCRIPTION), (tinea pedis...","[(tinea pedis, p_at, 2-week follow-up), (tinea..."
14404,5,Cross-linked hemoglobin solution as a resuscit...,"[(Cross-linked hemoglobin solution, DETAILED_D...","[(less than 40% of pre, returned_to, Cross-lin..."
14409,3,The supraspinal anxiolytic effect of baclofen ...,"[(supraspinal, BIOLOGICAL_STRUCTURE), (anxioly...","[(twice per week, effect_for, reduction), (sta..."
